To open this notebook in Colab visit https://goo.gl/magenta/groovae-colab

<img src="https://magenta-staging.tensorflow.org/assets/groovae/score-groove.png" alt="GrooVAE Figure" >

#@title Setup Environment

print('Installing dependencies...')

#!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -U -q magenta

In [1]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

import copy, warnings, librosa, numpy as np
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Colab/Notebook specific stuff
import IPython.display

# Magenta specific stuff
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
#from magenta.models.music_vae import data
import note_seq
from note_seq import midi_synth
from note_seq.sequences_lib import concatenate_sequences
from note_seq.protobuf import music_pb2

/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present i

In [2]:
# If a sequence has notes at time before 0.0, scootch them up to 0
def start_notes_at_0(s):
  for n in s.notes:
      if n.start_time < 0:
          n.end_time -= n.start_time
          n.start_time = 0
  return s

# Some midi files come by default from different instrument channels
# Quick and dirty way to set midi files to be recognized as drums
def set_to_drums(ns):
  for n in ns.notes:
    n.instrument=9
    n.is_drum = True
    
def unset_to_drums(ns):
  for note in ns.notes:
    note.is_drum=False
    note.instrument=0
  return ns

# quickly change the tempo of a midi sequence and adjust all notes
def change_tempo(note_sequence, new_tempo):
  new_sequence = copy.deepcopy(note_sequence)
  ratio = note_sequence.tempos[0].qpm / new_tempo
  for note in new_sequence.notes:
    note.start_time = note.start_time * ratio
    note.end_time = note.end_time * ratio
  new_sequence.tempos[0].qpm = new_tempo
  return new_sequence

# Load some configs to be used later
dc_tap = configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity'].data_converter
dc_quantize = configs.CONFIG_MAP['groovae_2bar_humanize'].data_converter

# quick method for removing microtiming and velocity from a sequence
def get_quantized_2bar(s, velocity=0):
  new_s = dc_quantize.from_tensors(dc_quantize.to_tensors(s).inputs)[0]
  new_s = change_tempo(new_s, s.tempos[0].qpm)
  if velocity != 0:
    for n in new_s.notes:
      n.velocity = velocity
  return new_s

# quick method for turning a drumbeat into a tapped rhythm
def get_tapped_2bar(s, velocity=85, ride=False):
  new_s = dc_tap.from_tensors(dc_tap.to_tensors(s).inputs)[0]
  new_s = change_tempo(new_s, s.tempos[0].qpm)
  if velocity != 0:
    for n in new_s.notes:
      n.velocity = velocity
  if ride:
    for n in new_s.notes:
      n.pitch = 42
  return new_s

# Calculate quantization steps but do not remove microtiming
def quantize(s, steps_per_quarter=4):
  return note_seq.sequences_lib.quantize_note_sequence(s,steps_per_quarter)

# Destructively quantize a midi sequence
def flatten_quantization(s):
  beat_length = 60. / s.tempos[0].qpm
  step_length = beat_length / 4 #s.quantization_info.steps_per_quarter
  new_s = copy.deepcopy(s)
  for note in new_s.notes:
    note.start_time = step_length * note.quantized_start_step
    note.end_time = step_length * note.quantized_end_step
  return new_s

# Calculate how far off the beat a note is
def get_offset(s, note_index):
  q_s = flatten_quantization(quantize(s))
  true_onset = s.notes[note_index].start_time
  quantized_onset = q_s.notes[note_index].start_time
  diff = quantized_onset - true_onset
  beat_length = 60. / s.tempos[0].qpm
  step_length = beat_length / 4#q_s.quantization_info.steps_per_quarter
  offset = diff/step_length
  return offset

def is_4_4(s):
  ts = s.time_signatures[0]
  return (ts.numerator == 4 and ts.denominator ==4)

#def preprocess_4bar(s):
#  return dc_4bar.from_tensors(dc_4bar.to_tensors(s).outputs)[0]

def preprocess_2bar(s):
  return dc_quantize.from_tensors(dc_quantize.to_tensors(s).outputs)[0]

#def _slerp(p0, p1, t):
#  """Spherical linear interpolation."""
#  omega = np.arccos(np.dot(np.squeeze(p0/np.linalg.norm(p0)),
#    np.squeeze(p1/np.linalg.norm(p1))))
#  so = np.sin(omega)
#  return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

In [3]:
#print('Downloading drum samples...')
# Download a drum kit for playing drum midi
#!gsutil -q -m cp gs://magentadata/soundfonts/Standard_Drum_Kit.sf2 .

print("Download MIDI data...")

# Load MIDI files from GMD with MIDI only (no audio) as a tf.data.Dataset
dataset_2bar = tfds.as_numpy(tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.VALIDATION,
    try_gcs=True))

dev_sequences = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_2bar]
_ = [set_to_drums(s) for s in dev_sequences]
dev_sequences = [s for s in dev_sequences if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]

print(len(dev_sequences))

Download MIDI data...


2022-03-15 12:54:28.941066: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".
2022-03-15 12:54:36.029179: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-15 12:54:36.059523: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-15 12:54:36.059545: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Can

2039


In [8]:
for features in dataset_2bar:
    x=note_seq.midi_to_note_sequence(features["midi"])
    break

In [9]:
x

ticks_per_quarter: 480
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 180.00018000018
}
notes {
  pitch: 36
  velocity: 127
  start_time: 0.14652763125
  end_time: 0.247221975
  is_drum: true
}
notes {
  pitch: 55
  velocity: 127
  start_time: 0.3402774375
  end_time: 0.44027733750000003
  is_drum: true
}
notes {
  pitch: 38
  velocity: 127
  start_time: 0.34305521250000004
  end_time: 0.44374955625
  is_drum: true
}
notes {
  pitch: 36
  velocity: 127
  start_time: 0.6631937812500001
  end_time: 0.7645825687500001
  is_drum: true
}
notes {
  pitch: 26
  velocity: 127
  start_time: 0.6819437625
  end_time: 0.7826381062500001
  is_drum: true
}
notes {
  pitch: 40
  velocity: 127
  start_time: 0.9861101250000001
  end_time: 1.0874989125
  is_drum: true
}
notes {
  pitch: 26
  velocity: 127
  start_time: 0.9951378937500001
  end_time: 1.0972211250000001
  is_drum: true
}
notes {
  pitch: 36
  velocity: 127
  start_time: 1.16110995
  end_time: 1.2624

In [7]:
features['midi']

b"MThd\x00\x00\x00\x06\x00\x01\x00\x02\x01\xe0MTrk\x00\x00\x00\x19\x00\xffQ\x03\x05\x16\x15\x00\xffX\x04\x04\x02\x18\x08\x00\xffY\x02\x00\x00\x01\xff/\x00MTrk\x00\x00\x00\xe0\x00\xff\x03\nMidi Drums\x00\xc9\x00\x00\xb9\x04$\x1e\x04\x1f+\x04\x1a6\x04\x15@\x04\x10\x14\x99$\x7f,\xb9\x04\x0ce\x99$\x00\x81\x067\x7f\x04&\x7f\x81\x0c7\x00\x05&\x00\x08\xb9\x04\x07\x81P\x04\x0bd\x99$\x7f\x1a\xb9\x04\x0c\x01\x99\x1a\x7fl\xb9\x04\x07\x0b\x99$\x00\x1a\x1a\x00\x82%(\x7f\r\xb9\x04\x04\x00\x99\x1a\x7f\x81\x05(\x00\x0e\x1a\x00\\$\x7f\x81\x12$\x00o\xb9\x04\x04\x00\x99\x1a\x7f\x81\x12\x1a\x00'$\x7fJ\xb9\x04\x00H\x99$\x00|(\x7f\x0f\xb9\x04\x02\x00\x99\x1a\x7fM\xb9\x04\x005\x99(\x00\x10\x1a\x00\x81?\xb9\x04\x04M\x04\tD\x04\t\x00\x99\x1a\x7f\x11$\x7fT\xb9\x04\x04-\x99\x1a\x00\x10$\x00\x825\xb9\x04\x04\x00\x99\x1a\x7f\t(|\x81\t\x1a\x00\x07(\x00S$\x7f\x81\x11$\x00b\xb9\x04\x07\x01\xff/\x00"

ticks_per_quarter
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 180.00018000018
}
notes {
}
total_time
control_changes {
}

In [28]:
dev_sequences[1111]

ticks_per_quarter: 480
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 108.000108000108
}
notes {
  pitch: 38
  velocity: 124
  start_time: 0.13310171875000001
  end_time: 0.2337960625
  instrument: 9
  is_drum: true
  quantized_start_step: 1
  quantized_end_step: 2
}
notes {
  pitch: 36
  velocity: 127
  start_time: 0.2523145625
  end_time: 0.35300890625000003
  instrument: 9
  is_drum: true
  quantized_start_step: 2
  quantized_end_step: 3
}
notes {
  pitch: 26
  velocity: 127
  start_time: 0.25347196875
  end_time: 0.35532371875
  instrument: 9
  is_drum: true
  quantized_start_step: 2
  quantized_end_step: 3
}
notes {
  pitch: 44
  velocity: 67
  start_time: 0.50925875
  end_time: 0.60995309375
  instrument: 9
  is_drum: true
  quantized_start_step: 4
  quantized_end_step: 5
}
notes {
  pitch: 38
  velocity: 127
  start_time: 0.541666125
  end_time: 0.643517875
  instrument: 9
  is_drum: true
  quantized_start_step: 4
  quantized_end_step: 5
}

print("Loading model checkpoints...")

# Download all the models
#!gsutil -q -m cp gs://magentadata/models/music_vae/checkpoints/groovae_*.tar .
!gsutil -q -m cp gs://magentadata/models/music_vae/checkpoints/groovae_2bar_tap_fixed_velocity.tar .
#GROOVAE_4BAR = "groovae_4bar.tar"
#GROOVAE_2BAR_HUMANIZE = "groovae_2bar_humanize.tar"
#GROOVAE_2BAR_HUMANIZE_NOKL = "groovae_2bar_humanize_nokl.tar"
#GROOVAE_2BAR_HITS_CONTROL = "groovae_2bar_hits_control.tar"
GROOVAE_2BAR_TAP_FIXED_VELOCITY = "groovae_2bar_tap_fixed_velocity.tar"
#GROOVAE_2BAR_ADD_CLOSED_HH = "groovae_2bar_add_closed_hh.tar"
#GROOVAE_2BAR_HITS_CONTROL_NOKL = "groovae_2bar_hits_control_nokl.tar"

#print("Downloading audio data...")
#!gsutil -q -m cp gs://magentadata/models/music_vae/groovae_colab/*wav .

In [5]:
GROOVAE_2BAR_TAP_FIXED_VELOCITY = "groovae_2bar_tap_fixed_velocity.tar"

def play(note_sequence, sf2_path='Standard_Drum_Kit.sf2'):
  if sf2_path:
    audio_seq = midi_synth.fluidsynth(start_notes_at_0(note_sequence), sample_rate=44100, sf2_path=sf2_path)
    IPython.display.display(IPython.display.Audio(audio_seq, rate=44100))
  else:
    note_seq.play_sequence(start_notes_at_0(note_sequence), synth=note_seq.fluidsynth)

def download(note_sequence, filename):
  note_seq.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)
  
def download_audio(audio_sequence, filename, sr):
  librosa.output.write_wav(filename, audio_sequence, sr=sr, norm=True)
  files.download(filename)

# Tap2Drum: Generate a beat from any rhythm 

While the Groove model works by removing the micro-timing and velocity information and learning to predict them from just the drum pattern, we can also go in the opposite direction.  Here, we take a representation of a Groove as input (in the form of a rhythm that can have precise timing but where drum categories are ignored) - and then generate drum beats that match the groove implied by this rhythm.  We trained this model by collapsing all drum hits from each beat in the training data to a single "tapped" rhythm, and then learning to decode full beats from that rhythm.  This allows us to input any rhythm we like through the precise onset timings in a "tap" and let the model decode our rhythm into a beat. We can even simply record taps as audio, or extract them from a recording of another instrument, rather than needing a midi controller.

In [8]:
def mix_tracks(y1, y2, stereo = False):
  l = max(len(y1),len(y2))
  y1 = librosa.util.fix_length(y1, l)
  y2 = librosa.util.fix_length(y2, l)
  
  if stereo:
    return np.vstack([y1, y2])  
  else:
    return y1+y2

def make_click_track(s):
  last_note_time = max([n.start_time for n in s.notes])
  beat_length = 60. / s.tempos[0].qpm 
  i = 0
  times = []
  while i*beat_length < last_note_time:
    times.append(i*beat_length)
    i += 1
  return librosa.clicks(times)

def drumify(s, model, temperature=1.0): 
  encoding, mu, sigma = model.encode([s])
  decoded = model.decode(encoding, length=32, temperature=temperature)
  return decoded[0]

def combine_sequences(seqs):
  # assumes a list of 2 bar seqs with constant tempo
  for i, seq in enumerate(seqs):
    shift_amount = i*(60 / seqs[0].tempos[0].qpm * 4 * 2)
    if shift_amount > 0:
      seqs[i] = note_seq.sequences_lib.shift_sequence_times(seq, shift_amount)
  return note_seq.sequences_lib.concatenate_sequences(seqs)

def combine_sequences_with_lengths(sequences, lengths):
  seqs = copy.deepcopy(sequences)
  total_shift_amount = 0
  for i, seq in enumerate(seqs):
    if i == 0:
      shift_amount = 0
    else:
      shift_amount = lengths[i-1]
    total_shift_amount += shift_amount
    if total_shift_amount > 0:
      seqs[i] = note_seq.sequences_lib.shift_sequence_times(seq, total_shift_amount)
  combined_seq = music_pb2.NoteSequence()
  for i in range(len(seqs)):
    tempo = combined_seq.tempos.add()
    tempo.qpm = seqs[i].tempos[0].qpm
    tempo.time = sum(lengths[0:i-1])
    for note in seqs[i].notes:
      combined_seq.notes.extend([copy.deepcopy(note)])
  return combined_seq

def get_audio_start_time(y, sr):
  tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
  beat_times = librosa.frames_to_time(beat_frames, sr=sr)
  onset_times = librosa.onset.onset_detect(y, sr, units='time')
  start_time = onset_times[0] 
  return start_time

def audio_tap_to_note_sequence(f, velocity_threshold=30):
  y, sr = librosa.load(f)
  # pad the beginning to avoid errors with onsets right at the start
  y = np.concatenate([np.zeros(1000),y])
  tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
  # try to guess reasonable tempo
  beat_times = librosa.frames_to_time(beat_frames, sr=sr)
  onset_frames = librosa.onset.onset_detect(y, sr, units='frames')
  onset_times = librosa.onset.onset_detect(y, sr, units='time')
  start_time = onset_times[0]
  onset_strengths = librosa.onset.onset_strength(y, sr)[onset_frames]
  normalized_onset_strengths = onset_strengths / np.max(onset_strengths)
  onset_velocities = np.int32(normalized_onset_strengths * 127)
  note_sequence = music_pb2.NoteSequence()
  note_sequence.tempos.add(qpm=tempo)
  for onset_vel, onset_time in zip(onset_velocities, onset_times):
    if onset_vel > velocity_threshold and onset_time >= start_time:  # filter quietest notes
      note_sequence.notes.add(
        instrument=9, pitch=42, is_drum=True,
        velocity=onset_vel,  # use fixed velocity here to avoid overfitting
        start_time=onset_time - start_time,
        end_time=onset_time - start_time)

  return note_sequence

# Allow encoding of a sequence that has no extracted examples
# by adding a quiet note after the desired length of time
def add_silent_note(note_sequence, num_bars):
  tempo = note_sequence.tempos[0].qpm
  length = 60/tempo * 4 * num_bars
  note_sequence.notes.add(
    instrument=9, pitch=42, velocity=0, start_time=length-0.02, 
    end_time=length-0.01, is_drum=True)
  
def get_bar_length(note_sequence):
  tempo = note_sequence.tempos[0].qpm
  return 60/tempo * 4

def sequence_is_shorter_than_full(note_sequence):
  return note_sequence.notes[-1].start_time < get_bar_length(note_sequence)

def make_tap_sequence(tempo, onset_times, onset_frames, onset_velocities,
                     velocity_threshold, start_time, end_time):
  note_sequence = music_pb2.NoteSequence()
  note_sequence.tempos.add(qpm=tempo)
  for onset_vel, onset_time in zip(onset_velocities, onset_times):
    if onset_vel > velocity_threshold and onset_time >= start_time and onset_time < end_time:  # filter quietest notes
      note_sequence.notes.add(
        instrument=9, pitch=42, is_drum=True,
        velocity=onset_vel,  # model will use fixed velocity here
        start_time=onset_time - start_time,
        end_time=onset_time -start_time + 0.01
      )
  return note_sequence

Here are a couple of examples using MIDI rhythms:

In [9]:
config_2bar_tap = configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity']
groovae_2bar_tap = TrainedModel(config_2bar_tap, 1, checkpoint_dir_or_path=GROOVAE_2BAR_TAP_FIXED_VELOCITY)

sequence_indices = [1111, 366]
for i in sequence_indices:
  s = start_notes_at_0(dev_sequences[i])
  s = change_tempo(get_tapped_2bar(s, velocity=85, ride=True), dev_sequences[i].tempos[0].qpm)
  print("\nPlaying Tapped Beat: ")
  #play(start_notes_at_0(s))
  h = change_tempo(drumify(s, groovae_2bar_tap), s.tempos[0].qpm)
  print("Playing Drummed Beat: ")
  #play(start_notes_at_0(h))

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]



INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]



INFO:tensorflow:
Decoder Cells:
  units: [256, 256]



...

INFO:tensorflow:Unbundling checkpoint.


INFO:tensorflow:Unbundling checkpoint.


INFO:tensorflow:Restoring parameters from /tmp/tmpatrrta_i/groovae_2bar_tap_fixed_velocity/model.ckpt-3668


INFO:tensorflow:Restoring parameters from /tmp/tmpatrrta_i/groovae_2bar_tap_fixed_velocity/model.ckpt-3668



Playing Tapped Beat: 
Playing Drummed Beat: 

Playing Tapped Beat: 
Playing Drummed Beat: 


In [15]:
slt="0. 0.03 120. 0.0625 0.0925 0. 0.125 0.155 0. 0.1875 0.2175 0. 0.25 0.28 0. 0.3125 0.3425 0. 0.375 0.405 0. 0.4375 0.4675 0. 0.5 0.53 0. 0.5625 0.5925 0. 0.625 0.655 0. 0.6875 0.7175 0. 0.75 0.78 0. 0.8125 0.8425 0. 0.875 0.905 0. 0.9375 0.9675 0. 1. 1.03 120. 1.0625 1.0925 0. 1.125 1.155 0. 1.1875 1.2175 0. 1.25 1.28 0. 1.3125 1.3425 0. 1.375 1.405 0. 1.4375 1.4675 0. 1.5 1.53 0. 1.5625 1.5925 0. 1.625 1.655 0. 1.6875 1.7175 0. 1.75 1.78 0. 1.8125 1.8425 0. 1.875 1.905 0. 1.9375 1.9675 0.".split(' ')

In [22]:
midi_array=[]
for i in range((len(slt)//3)-1):
    #print(slt[3*i:3*(i+1)])
    start=float(slt[3*i])
    end=float(slt[3*i+1])
    vel=float(slt[3*i+2])
    midi_array.append([start,end,vel])
midi_array=np.array(midi_array)

In [24]:
midi_array

array([[0.0000e+00, 3.0000e-02, 1.2000e+02],
       [6.2500e-02, 9.2500e-02, 0.0000e+00],
       [1.2500e-01, 1.5500e-01, 0.0000e+00],
       [1.8750e-01, 2.1750e-01, 0.0000e+00],
       [2.5000e-01, 2.8000e-01, 0.0000e+00],
       [3.1250e-01, 3.4250e-01, 0.0000e+00],
       [3.7500e-01, 4.0500e-01, 0.0000e+00],
       [4.3750e-01, 4.6750e-01, 0.0000e+00],
       [5.0000e-01, 5.3000e-01, 0.0000e+00],
       [5.6250e-01, 5.9250e-01, 0.0000e+00],
       [6.2500e-01, 6.5500e-01, 0.0000e+00],
       [6.8750e-01, 7.1750e-01, 0.0000e+00],
       [7.5000e-01, 7.8000e-01, 0.0000e+00],
       [8.1250e-01, 8.4250e-01, 0.0000e+00],
       [8.7500e-01, 9.0500e-01, 0.0000e+00],
       [9.3750e-01, 9.6750e-01, 0.0000e+00],
       [1.0000e+00, 1.0300e+00, 1.2000e+02],
       [1.0625e+00, 1.0925e+00, 0.0000e+00],
       [1.1250e+00, 1.1550e+00, 0.0000e+00],
       [1.1875e+00, 1.2175e+00, 0.0000e+00],
       [1.2500e+00, 1.2800e+00, 0.0000e+00],
       [1.3125e+00, 1.3425e+00, 0.0000e+00],
       [1.